# Pandas CSV file exploration and cleaning

In [1]:
# Importing all libraries
import warnings
import pandas as pd 
import sqlite3

#Ignoring ugly warnings
warnings.filterwarnings("ignore")

In [2]:
# Creating Multiple DataFrames to explore and clean each individual file
df_gross = pd.read_csv('../zippedData/bom.movie_gross.csv.gz')
df_budgets = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')

In [3]:
# Checking info
df_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [4]:
# Checking info
df_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


df_budgets and df_gross are the only CSV files I believe to be useful, the others either have irrelvant data to our EDA or are already in our SQL database.

Time to dive deeper into the data we have in our CSV files so we can understand it better so that We can prepare the data to make business recomendations

In [5]:
# Converting to interger and date time to be used for comparision and visualizations
df_budgets['production_budget'] = df_budgets['production_budget'].str.replace('[$,]', '', regex=True).astype(int)
df_budgets['domestic_gross'] = df_budgets['domestic_gross'].str.replace('[$,]', '', regex=True).astype(int)
df_budgets['worldwide_gross'] = df_budgets['worldwide_gross'].str.replace('[$,]', '', regex=True).astype(int)
df_budgets['release_date'] = pd.to_datetime(df_budgets['release_date'], format='%b %d, %Y')

In [6]:
df_budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
...,...,...,...,...,...,...
5777,78,2018-12-31,Red 11,7000,0,0
5778,79,1999-04-02,Following,6000,48482,240495
5779,80,2005-07-13,Return to the Land of Wonders,5000,1338,1338
5780,81,2015-09-29,A Plague So Pleasant,1400,0,0


In [7]:
# Converting gross
df_gross['foreign_gross'] = df_gross['foreign_gross'].str.replace('[$,]', '', regex=True).astype(float)
df_gross

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


# SQL Database exploration and cleaning

In [8]:
# Printing the Schema of the Database
conn = sqlite3.connect('../zippedData/im.db')
pd.read_sql("""
    SELECT name
    FROM sqlite_master
    WHERE type == 'table'
""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [9]:
# We should get the information regarding each table so we can see what were working with
# Retrieve the list of tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Looping through the tables and get info for each table
for table in tables:
    table_name = table[0]
    
    # Query the table and create a DataFrame
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    
    # Display info for the DataFrame
    print(f"Table Name: {table_name}")
    print(df.info())
    print('\n')

Table Name: movie_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None


Table Name: directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB
None


Table Name: known_for
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 e

Most of these tables are pretty straight foward however it would be good to look into anything that isnt as clear before we make any major changes to our database

In [10]:
pd.read_sql("""
    SELECT *
    FROM movie_akas
""", conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [11]:
pd.read_sql("""
    SELECT *
    FROM principals
""", conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [12]:
pd.read_sql("""
    SELECT *
    FROM known_for
""", conn)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
...,...,...
1638255,nm9990690,tt9090932
1638256,nm9990690,tt8737130
1638257,nm9991320,tt8734436
1638258,nm9991320,tt9615610


Some of these tables and columns are irrelevant to us, We should drop these. Theres no reason to take up more space than necessary and prolong any potential load times.

In [13]:
#Dropping unnecessary columns and tables
cursor.execute("DROP TABLE movie_akas")
cursor.execute("ALTER TABLE persons DROP COLUMN death_year;")
cursor.execute("DROP TABLE principals")

In [15]:
# Close Connection and Save CSV file
conn.close()
df_gross.to_csv('../zippedData/gross_movie_cleaned.csv')
df_budgets.to_csv('../zippedData/movie_budget_cleaned.csv')